In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#Determine the number of players
player = len(purchase_data["SN"].unique())
player_num = pd.DataFrame({"Total Players": [player]})
player_num

,Total Players
0,576


In [3]:
#Find Purchasing Data

#Unique Items
items=len(purchase_data["Item ID"].unique())

#Average Price of Items Purchased
avg_price=purchase_data["Price"].mean()

#Number of Purchases
num_purchase=len(purchase_data["Price"])

#Total Revenue from Purchases
total_rev=purchase_data["Price"].sum()

#Create DataFrame
calc_df=pd.DataFrame({"Number of Unique Items":[items],
                      "Average Price": [avg_price],
                      "Number of Purchases": [num_purchase],
                      "Total Revenue": [total_rev]})
#Change format
calc_df = calc_df.style.format({"Average Price": "${:,.2f}",
                                "Total Revenue": "${:,.2f}"})
calc_df



,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
#Determine Gender Demographics

#Create Dataframe for analysis from total data
unique_player=pd.DataFrame({"SN":purchase_data["SN"], 
                            "Gender":purchase_data["Gender"],
                            "Age": purchase_data["Age"]})
unique_player=unique_player.drop_duplicates().reset_index(drop=True)

#Calculate number of users per gender
count=unique_player["Gender"].value_counts()

#Build Output Dataframe
gender_dem=pd.DataFrame({"Total Count": count,"Percentage of Players": count*100/player})

#Formatting
gender_dem["Percentage of Players"] = gender_dem["Percentage of Players"].map("{:,.2f}%".format)
gender_dem

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [5]:
#Purchasing Analysis by Gender

#Create Dataframe for analysis from total data
players=pd.DataFrame({"Gender":purchase_data["Gender"],
                      "Price": purchase_data["Price"]})

#Group by gender and calculate
purchase_count=players.groupby(["Gender"]).count()
avg_purch_price=players.groupby(["Gender"]).mean()
tot_purch_price=players.groupby(["Gender"]).sum()

#create gender counts dataframe from previous calculation
gender_counts=pd.DataFrame({"Gender Counts": count})
gender_counts['labels']=gender_counts.index.str.lower()
gender_counts = gender_counts.sort_values('labels').drop('labels',axis=1)

#Build output dataframe
gender_purchase=pd.DataFrame({"Purchase Count": purchase_count["Price"],
                              "Average Purchase Price": avg_purch_price["Price"],
                              "Total Purchase Price": tot_purch_price["Price"],
                              "Avg Total Purchase per Person":tot_purch_price["Price"]/gender_counts["Gender Counts"]})

#Formatting
gender_purchase["Average Purchase Price"]=gender_purchase["Average Purchase Price"].map("${:,.2f}".format)
gender_purchase["Total Purchase Price"]=gender_purchase["Total Purchase Price"].map("${:,.2f}".format)
gender_purchase["Avg Total Purchase per Person"]=gender_purchase["Avg Total Purchase per Person"].map("${:,.2f}".format)
gender_purchase

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [6]:
#Age Demographics

#Labelling bins
bins_cuts=[0,9,14,19,24,29,34,39,1000]
bin_labels=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

unique_player['Age-Binned']=pd.cut(unique_player['Age'], bins=bins_cuts, labels=bin_labels)

bins=unique_player["Age-Binned"].value_counts()
bins=bins.sort_index()

#Calculate numbers per bin
for x in range(len(bins)):
    z=pd.DataFrame({"Total Count": [bins[x]], "Percentage of Players": [bins[x]/player]},
                    index=[bin_labels[x]])
    if x==0:
        Ages=z
    else:
        Ages=Ages.append(z)

Ages=Ages.style.format({"Percentage of Players": "{:.2%}"})

Ages

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [7]:
#Purchasing Analysis by Age

#Create Dataframe for calculations
purch_age=pd.DataFrame({"Age":purchase_data["Age"],
                        "Price": purchase_data["Price"]})
#Re-bin new dataframe
purch_age['Age-Binned']=pd.cut(purch_age['Age'], bins=bins_cuts, labels=bin_labels)

#Calculate values through groupby
p_count=purch_age.groupby(["Age-Binned"]).count()
p_mean=purch_age.groupby(["Age-Binned"]).mean()
p_sum=purch_age.groupby(["Age-Binned"]).sum()

#Build output dataframe
Purchase_age=pd.DataFrame({"Purchase Count": p_count["Price"],
                           "Average Purchase Price": p_mean["Price"],
                           "Total Purchase Value": p_sum["Price"],
                           "Avg Total Purchase per Person": p_sum["Price"]/p_count["Price"]})

#Formatting
Purchase_age.index.name="Age Ranges"
Purchase_age["Average Purchase Price"]=Purchase_age["Average Purchase Price"].map("${:,.2f}".format)
Purchase_age["Total Purchase Value"]=Purchase_age["Total Purchase Value"].map("${:,.2f}".format)
Purchase_age["Avg Total Purchase per Person"]=Purchase_age["Avg Total Purchase per Person"].map("${:,.2f}".format)
Purchase_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,"$1,114.06",$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


In [8]:
#Top Spenders List

#Create Dataframe for analysis
spender_list=pd.DataFrame({"SN":purchase_data["SN"],
                           "Price": purchase_data["Price"]})

#Calculate values
spend_count=spender_list.groupby("SN").count()
spend_mean=spender_list.groupby("SN").mean()
spend_sum=spender_list.groupby("SN").sum()

#Build output DataFrame
top_spender=pd.DataFrame({"Purchase Count": spend_count["Price"],
                          "Average Purchase Price": spend_mean["Price"],
                          "Total Purchase Value": spend_sum["Price"]})

#Formatting
top_spender=top_spender.sort_values("Total Purchase Value", ascending=False)
top_spender["Average Purchase Price"]=top_spender["Average Purchase Price"].map("${:,.2f}".format)
top_spender["Total Purchase Value"]=top_spender["Total Purchase Value"].map("${:,.2f}".format)

top_spender.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [9]:
#Most Popular Items

#Create Dataframe for analysis
item=pd.DataFrame({"Item Name": purchase_data["Item Name"],
                   "Item ID": purchase_data["Item ID"],
                   "Price": purchase_data["Price"]})

#Calculate values
item_count=item.groupby(["Item ID","Item Name"]).count()
item_mean=item.groupby(["Item ID","Item Name"]).mean()
item_sum=item.groupby(["Item ID","Item Name"]).sum()

#Build output DataFrame
item_popular=pd.DataFrame({"Purchase Count":item_count["Price"],
                           "Item Price": item_mean["Price"],
                           "Total Purchase Value": item_sum["Price"]})

#Formatting
item_popular=item_popular.sort_values(["Purchase Count","Item ID"], ascending=False)
item_profit=item_popular.sort_values(["Total Purchase Value"], ascending=False) #Save before formatting for next part
item_popular["Item Price"]=item_popular["Item Price"].map("${:,.2f}".format)
item_popular["Total Purchase Value"]=item_popular["Total Purchase Value"].map("${:,.2f}".format)
item_popular.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [10]:
#Most Profitable Items

#Formatting
item_profit["Item Price"]=item_profit["Item Price"].map("${:,.2f}".format)
item_profit["Total Purchase Value"]=item_profit["Total Purchase Value"].map("${:,.2f}".format)
item_profit.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
